In [71]:

from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationBufferWindowMemory(
    llm=llm,
    k=6,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

examples = [
    {"movie":"Finding Nemo", "answer":"🐠🌊🔍"},
    {"movie":"Kung Fu Panda", "answer":"🐼🥋🎍"},
    {"movie":"The Terminator", "answer":"🤖🔫🕶️"},
    ]

example_prompt = ChatPromptTemplate.from_messages([
    ("human","{movie}를 3개의 이모지로 표현해줘."), ("ai","{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a movie expert who can get to the heart of a movie's plot and express it in three emojis."),
    example_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

invoke_chain("The Lord Of The Rings")
invoke_chain("avatar2")
invoke_chain("Home alone")

content='🧝🧙🗡️'
content='🪐🌿🌊'
content='🏠😱🎄'


In [72]:
invoke_chain("Of the movies I've asked you about, tell me which ones have dwarves in them. ")

content='"The Lord of the Rings" has dwarves in it.'
